# Nat Scott

## Research question/interests

**Is there a correlation between political alignment & living in neighbourhoods with large quantities of LGBT people?** The obvious answer to this question is "yes, they are going to mostly be democrats" but anyone who's ever been around queer people will know that this question is quite a bit more nuanced than that, and this nuance is what we hope to capture in investigating this question.

- The gaybourhoods data set does not include data on residents political alignments, however, there is a wealth of electoral data available freely online that we intend on incorporating into this project. The primary difficulty then will be developing a geographic "compatibility layer" between the data sets so that the data can be understood in the same context. To build this, we intend on working with the OpenStreetMap API to create an additional column representing observations position space in a more neutral way, such as their coordinates.
- Alternatively, we've also considered working with an additional data set that links US zip codes to their longitude and lattitude positions. As such, incorporating this data would be as easy as merging the two tables.


**Is there a correlation between geographical stratums & being LGBT?** This question is more abstract, and will serve as a preliminary exploration of the data in hopes of establishing two key details along the way that will shape the rest of the project: how do we quantify queerness, and how do we best represent it visually?

- Once again, representing this data visually will require determining the coordinates associated with each observation.
- The gaybourhoods data set defines a "gaybourhood index" which effectively measures how friendly a given neighbourhood is to queer people. Since this index is entirely subjective, we will need to closely evaluate it's usefulness for our project and investigate different ways to quantify "queer-friendliness"
- In addition to the last point, since, of course, no matter what choice of observations we make, the measurement will still be subjective, answering this research question will come more so in the form of comparing and contrasting different measurements to see what they tell us.
- Obviously, visualizing this among many aspects of the other research questions would involve projecting the data onto a map of the United States, so visualizing this research question would motivate many of the visualizations for other components of this project

In [50]:
import pandas as pd
import seaborn as sns

## Data Wrangling

In [76]:
## counties - Relating US counties to their long/lat position on the Earth
counties = pd.read_csv("../data/raw/us-county-boundaries.csv", sep=";")

counties = counties.rename({
    "NAME": "name",
    "INTPTLAT": "lat",
    "INTPTLON": "long",
}, axis="columns")

# Combine the county name with the state code
def combine_name_state(row):
    row["name"] = f"{row['name']} {row['STUSAB']}"
    return row

counties = counties.apply(combine_name_state, axis="columns")

# We don't need this column anymore
counties = counties.drop(["STUSAB"], axis="columns")

counties.to_csv("../data/processed/us-county-boundaries.csv")
counties.head()

,name,lat,long
0,Hancock OH,41.000471,-83.666033
1,Stafford VA,38.413261,-77.451334
2,Webster NE,40.180646,-98.498590
3,Dimmit TX,28.423587,-99.765871
4,Cedar IA,41.772360,-91.132610


In [81]:
## pol - Election results from the 2012 American presidential election
pol = pd.read_csv("../data/raw/countypres_2000-2020.csv")

# We only want 2012--the latest election before the gb data was collected

pol = pol[pol["year"] == 2012].reset_index()

# Get rid of undesireable columns
pol = pol.drop([
    "year", "state", "county_fips", "office",
    "candidate", "version", "mode", "index",
], axis="columns")

# Change the column names to make them a little more friendly
pol.rename({
    "county_name": "county",
    "state_po": "state",
    "candidatevotes": "votes",
    "totalvotes": "total"
}, axis="columns", inplace=True)

# Make cells lowercase
pol["county"] = pol["county"].apply(lambda x: x.capitalize())
pol["party"] = pol["party"].apply(lambda x: x.capitalize())

# Combine the county name with the state code
def combine_name_state(row):
    row["county"] = f"{row['county']} {row['state']}"
    return row

pol = pol.apply(combine_name_state, axis="columns")

# Add a percent column which will be useful when graphing
pol["percent"] = pol["votes"] / pol["total"]

# Attach long/lat data to each row
pol = pol.merge(counties, left_on="county", right_on="name")

# Now we can get rid of the state columns
pol = pol.drop(["state", "name"], axis="columns")

pol.to_csv("../data/processed/election-2012.csv", index=False)
pol.head()

,county,party,votes,total,percent,lat,long
0,Autauga AL,Democrat,6363,23932,0.265878,32.532237,-86.646439
1,Autauga AL,Republican,17379,23932,0.726183,32.532237,-86.646439
2,Autauga AL,Other,190,23932,0.007939,32.532237,-86.646439
3,Baldwin AL,Democrat,18424,85338,0.215894,30.659218,-87.746067
4,Baldwin AL,Republican,66016,85338,0.773583,30.659218,-87.746067


In [87]:
## gb - the gaybourhoods dataset
gb = pd.read_csv("../data/raw/gaybourhoods.csv")
cords = pd.read_csv("../data/raw/zip_lat_long.csv")

# Let's add long/lat columns to gb
gb = gb.merge(cords, left_on="GEOID10", right_on="ZIP")

# Get rid of unneeded columns
gb = gb.drop([
    "Mjoint_MF", "Mjoint_SS", "Mjoint_FF", "Mjoint_MM",
    "Cns_TotHH", "Cns_UPSS", "Cns_UPFF", "Cns_UPMM",
    "ParadeFlag", "FF_Tax", "FF_Cns", "MM_Tax", "MM_Cns",
    "SS_Index_Weight", "Parade_Weight", "Bars_Weight",
    "GEOID10", "ZIP",
], axis="columns")

# There's a lot of info baked into some of these columns. Especially the composite indexes.
# We'll leave their names as is for easy reference even if they're a little ugly.
gb = gb.rename({
    "LAT": "lat",
    "LNG": "long",
}, axis="columns")

gb.to_csv("../data/processed/gaybourhoods-nat.csv")
gb.head()

,Tax_Mjoint,TaxRate_SS,TaxRate_FF,TaxRate_MM,Cns_RateSS,Cns_RateFF,Cns_RateMM,CountBars,FF_Index,MM_Index,SS_Index,TOTINDEX,lat,long
0,2120,203.301887,28.773585,174.528302,77.125329,6.931719,70.193610,15,6.724415,48.288254,55.012669,67.077054,34.093828,-118.381697
1,5080,205.511811,33.464567,172.047244,88.478367,15.617404,72.860963,17,9.834048,48.578469,58.412517,61.866815,37.758057,-122.435410
2,5790,107.772021,16.753022,91.018998,46.771050,5.745582,41.025469,5,4.370779,26.360413,30.731192,37.908747,40.742039,-74.000620
3,3510,80.056980,21.082621,58.974359,31.619291,9.315448,22.303843,10,6.055939,15.939869,21.995808,37.530067,40.734012,-74.006746
4,2660,91.353383,12.781955,78.571429,21.763042,3.142678,18.620365,9,3.004058,18.280165,21.284224,35.843573,37.773134,-122.411167
